# Практическое занятие №2 - локализация объектов на изображениях

1. Разметка данных с помощью виджета QSL для Jupyter (https://github.com/faustomorales/qsl)
2. Знакомство с архитектурой нейронных сетей для локализации объектов на изображении.
3. Разработка простой системы управления по данным с видео-камеры.

In [4]:
!pip install -U -q qsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.1 MB/s eta 0:00:00


In [2]:
# These next two lines are only required on Google Colab.
# They are not needed in Jupyter Lab or Jupyter Notebook.
import google.colab
google.colab.output.enable_custom_widget_manager()

In [5]:
# Widgets have been tested on JupyterLab >= 3 and
# Jupyter Notebook >= 6
import qsl
import qsl.testing.files

In [4]:
qsl.testing.files.TEST_IMAGES[0]

'/usr/local/lib/python3.8/dist-packages/qsl/testing/data/image1.jpg'

In [30]:
%%writefile images.csv
target
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_009.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_010.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/image_011.jpg

Overwriting images.csv


In [6]:
import pandas

image_file = pandas.read_csv('images.csv')

In [ ]:
labeler = qsl.MediaLabeler(
    items=image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Object", "multiple": True, "options": [{"name": "bottle"}, {"name": "bug"}]}
        ]
    })
labeler.mode = "dark"
labeler

MediaLabeler(buttons={'prev': False, 'next': True, 'save': True, 'config': True, 'delete': False, 'ignore': Tr…

In [ ]:
# You can access the current state of the image labels at any time.
# Mostly self-explanatory. The segmentation maps provide map
# dimensions (width/height) and a run-length encoded representation
# with classes i (inside mask) and o (outside of mask). The mask
# may be lower-res than the image (you can tell from the dimensions)
# if the image was large, so you'll have to upsample and interpolate
# the map up to true image size. This little twist makes the in-browser
# behavior more memory efficient at the cost of mask precision.
with open('result.json', 'w') as file:
    file.write(str(labeler.items).replace("'", '"'))

In [ ]:
labels = None
with open('result.json', 'r') as file:
    labels = file.read()
    print(labels)

[{"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg", "labels": {"image": {}, "polygons": [{"points": [{"x": 0.642587170887295, "y": 0.3516193884040677}, {"x": 0.6700005413571787, "y": 0.3926353214413227}, {"x": 0.7643025357735784, "y": 0.45122951149454404}, {"x": 0.8235154159885271, "y": 0.5117768412162061}, {"x": 0.8728594828343176, "y": 0.5723241709378682}, {"x": 0.9046589925793826, "y": 0.5977149866275975}, {"x": 0.909045131854564, "y": 0.6191995229804453}, {"x": 0.934265432686857, "y": 0.6133401039751232}, {"x": 0.9638718727943313, "y": 0.6367777799964117}, {"x": 0.9265896889552896, "y": 0.7012313890549553}, {"x": 0.8893075051162478, "y": 0.7656849981134988}, {"x": 0.8706664131967269, "y": 0.8008415121454316}, {"x": 0.8114535329817782, "y": 0.8027946518138723}, {"x": 0.7686886750487598, "y": 0.785216394797906}, {"x": 0.7423718393976715, "y": 0.7754506964557024}, {"x": 0.7335995608473087, "y": 0.7344347634184474}, {"x": 0.725

In [11]:
%%writefile labels.json
[{"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg", "labels": {"image": {}, "polygons": [{"points": [{"x": 0.642587170887295, "y": 0.3516193884040677}, {"x": 0.6700005413571787, "y": 0.3926353214413227}, {"x": 0.7643025357735784, "y": 0.45122951149454404}, {"x": 0.8235154159885271, "y": 0.5117768412162061}, {"x": 0.8728594828343176, "y": 0.5723241709378682}, {"x": 0.9046589925793826, "y": 0.5977149866275975}, {"x": 0.909045131854564, "y": 0.6191995229804453}, {"x": 0.934265432686857, "y": 0.6133401039751232}, {"x": 0.9638718727943313, "y": 0.6367777799964117}, {"x": 0.9265896889552896, "y": 0.7012313890549553}, {"x": 0.8893075051162478, "y": 0.7656849981134988}, {"x": 0.8706664131967269, "y": 0.8008415121454316}, {"x": 0.8114535329817782, "y": 0.8027946518138723}, {"x": 0.7686886750487598, "y": 0.785216394797906}, {"x": 0.7423718393976715, "y": 0.7754506964557024}, {"x": 0.7335995608473087, "y": 0.7344347634184474}, {"x": 0.7259238171157413, "y": 0.7246690650762438}, {"x": 0.7204411430217645, "y": 0.6973251097180738}, {"x": 0.671097076175974, "y": 0.6816999923705481}, {"x": 0.6447802405248857, "y": 0.6992782493865146}, {"x": 0.607498056685844, "y": 0.730528484081566}, {"x": 0.5888569647663231, "y": 0.7539661601028546}, {"x": 0.5735054773031882, "y": 0.7539661601028546}, {"x": 0.5581539898400534, "y": 0.6973251097180738}, {"x": 0.5844708254911417, "y": 0.6367777799964117}, {"x": 0.6020153825918673, "y": 0.6133401039751232}, {"x": 0.604208452229458, "y": 0.5078705618793247}, {"x": 0.6228495441489789, "y": 0.42583869580481476}, {"x": 0.642587170887295, "y": 0.3516193884040677}], "labels": {"Object": ["bug"]}}, {"points": [{"x": 0.607498056685844, "y": 0.7422473220922102}, {"x": 0.6667109369007926, "y": 0.6914656907127518}, {"x": 0.6919312377330856, "y": 0.6953719700496331}, {"x": 0.6985104466458577, "y": 0.703184528723396}, {"x": 0.7050896555586298, "y": 0.703184528723396}, {"x": 0.7149584689277878, "y": 0.7227159254078032}, {"x": 0.7160550037465832, "y": 0.7383410427553289}, {"x": 0.7270203519345366, "y": 0.7481067410975324}, {"x": 0.7335995608473087, "y": 0.7715444171188209}, {"x": 0.7335995608473087, "y": 0.7813101154610246}, {"x": 0.7379857001224901, "y": 0.8086540708191945}, {"x": 0.731406491209718, "y": 0.8359980261773645}, {"x": 0.7292134215721273, "y": 0.8535762831933309}, {"x": 0.6919312377330856, "y": 0.9219361715887558}, {"x": 0.642587170887295, "y": 0.9473269872784851}, {"x": 0.6261391486053649, "y": 0.9356081492678409}, {"x": 0.6283322182429556, "y": 0.9512332666153666}, {"x": 0.6151738004174114, "y": 0.9649052442944516}, {"x": 0.5998223129542766, "y": 0.9707646632997737}, {"x": 0.5954361736790952, "y": 0.9863897806472994}, {"x": 0.5362232934641465, "y": 0.9902960599841808}, {"x": 0.518678736363421, "y": 0.9649052442944516}, {"x": 0.5044237837190815, "y": 0.9590458252891294}, {"x": 0.49894110962510474, "y": 0.9238893112571965}, {"x": 0.5066168533566722, "y": 0.9121704732465523}, {"x": 0.5077133881754675, "y": 0.8770139592146194}, {"x": 0.5230648756386024, "y": 0.8164666294929573}, {"x": 0.578988151397165, "y": 0.7695912774503803}, {"x": 0.5943396388602998, "y": 0.7734975567872617}, {"x": 0.607498056685844, "y": 0.7422473220922102}], "labels": {"Object": ["bottle"]}}], "masks": [], "dimensions": {"width": 1628, "height": 914}, "boxes": []}, "ignore": False}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg", "labels": {"image": {}, "polygons": [{"points": [{"x": 0.7392779362900288, "y": 0.4082576869024545}, {"x": 0.7657588936063751, "y": 0.4551328042493927}, {"x": 0.7999634634733225, "y": 0.5273986101592557}, {"x": 0.8330646601187554, "y": 0.5781799872851053}, {"x": 0.878302962200847, "y": 0.6602114426422472}, {"x": 0.9014737998526501, "y": 0.6953677806524508}, {"x": 0.9147142785108233, "y": 0.7363835083310216}, {"x": 0.9367817429411119, "y": 0.7520085474466677}, {"x": 0.9577458341498861, "y": 0.7461491577783005}, {"x": 0.9765031789156314, "y": 0.7539616773361235}, {"x": 0.9489188483777706, "y": 0.8008367946830617}, {"x": 0.9069906659602223, "y": 0.8691963408140131}, {"x": 0.8760962157578182, "y": 0.9316964972765973}, {"x": 0.8540287513275295, "y": 0.9902903939602701}, {"x": 0.8319612868972409, "y": 0.9883372640708143}, {"x": 0.7966533438087792, "y": 0.966852835286801}, {"x": 0.7381745630685144, "y": 0.9512277961711549}, {"x": 0.6807991555497639, "y": 0.9238839777187743}, {"x": 0.6454912124613021, "y": 0.9063058087136725}, {"x": 0.6256304944740424, "y": 0.886774509819115}, {"x": 0.614596762258898, "y": 0.8398993924721768}, {"x": 0.599149537157696, "y": 0.8281806131354422}, {"x": 0.5814955656134652, "y": 0.7598210670044907}, {"x": 0.583702312056494, "y": 0.7129459496575525}, {"x": 0.5737719530628641, "y": 0.6621645725317029}, {"x": 0.6134933890373836, "y": 0.6074769356269417}, {"x": 0.6488013321258455, "y": 0.5117735710436097}, {"x": 0.688522768100365, "y": 0.39067951789735267}, {"x": 0.6995565003155092, "y": 0.3750544787817066}, {"x": 0.7392779362900288, "y": 0.4082576869024545}], "labels": {"Object": ["bug"]}}, {"points": [{"x": 0.4314368074875025, "y": 0.6250558124389721}, {"x": 0.4512975254747623, "y": 0.6602121504491758}, {"x": 0.4579177648038489, "y": 0.6934153585699236}, {"x": 0.4744683631265653, "y": 0.7188060471328485}, {"x": 0.4943290811138251, "y": 0.7383373460274061}, {"x": 0.5009493204429116, "y": 0.7695874242586982}, {"x": 0.49653582755685394, "y": 0.7930249829321673}, {"x": 0.4855020953417096, "y": 0.8145094117161805}, {"x": 0.47115824346202206, "y": 0.8184156714950921}, {"x": 0.45681439158233444, "y": 0.8086500220478133}, {"x": 0.4512975254747623, "y": 0.7813062035954327}, {"x": 0.4424705397026468, "y": 0.7656811644797866}, {"x": 0.42812668782295926, "y": 0.7422436058063175}, {"x": 0.41709295560781495, "y": 0.7344310862484945}, {"x": 0.4016457305066129, "y": 0.6875559689015563}, {"x": 0.399438984063584, "y": 0.6543527607808085}, {"x": 0.41047271627872833, "y": 0.6250558124389721}, {"x": 0.4314368074875025, "y": 0.6250558124389721}, {"x": 0.4314368074875025, "y": 0.6250558124389721}], "labels": {"Object": ["bottle"]}}], "masks": [], "dimensions": {"width": 1625, "height": 918}, "boxes": []}, "ignore": False}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg"}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg"}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg"}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg"}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg"}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_009.jpg"}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_010.jpg"}, {"target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/image_011.jpg"}]

Writing labels.json


In [12]:
import json

labels = None
with open('labels.json', 'r') as file:
    labels = file.read()
labels = json.loads(labels.replace('True', 'true').replace('False', 'false'))

In [13]:
labeler = qsl.MediaLabeler(
    items=labels,
    
    #image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Type", "multiple": True, "options": [{"name": "bottle"}, {"name": "bag"}]}
        ]
    })
labeler.labels#  = labels
labeler.mode = "dark"
labeler

MediaLabeler(buttons={'prev': True, 'next': True, 'save': True, 'config': True, 'delete': False, 'ignore': Tru…

In [41]:
labeler.items

[{'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg',
  'labels': {'image': {'Type': ['bottle', 'bag']},
   'polygons': [{'points': [{'x': 0.5504597053803014, 'y': 0.768829345703125},
      {'x': 0.6952020980301018, 'y': 0.682891845703125},
      {'x': 0.7346772960255019, 'y': 0.729766845703125},
      {'x': 0.7555114283008519, 'y': 0.813751220703125},
      {'x': 0.7138431637501519, 'y': 0.930938720703125},
      {'x': 0.6425685007029016, 'y': 0.968048095703125},
      {'x': 0.6162517020393016, 'y': 0.966094970703125},
      {'x': 0.5822591704321515, 'y': 0.989532470703125},
      {'x': 0.5274325065496513, 'y': 0.985626220703125},
      {'x': 0.49343997494250125, 'y': 0.932891845703125},
      {'x': 0.5197567736061013, 'y': 0.827423095703125},
      {'x': 0.5504597053803014, 'y': 0.768829345703125}],
     'labels': {'Object': ['bottle']}}],
   'masks': [],
   'dimensions': {'width': 1628, 'height': 914},
   'boxes': []},
  'igno

In [21]:
image_file.to_dict(orient='records')

[{'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_009.jpg'},
 {'target': 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_010